In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from numba import njit,jit
from numba.typed import List 


In [8]:
@njit
def oscillatorModelOde(Y, t, can): 

    ka1,kb1,kcat1,ka2,kb2,ka3,kb3,ka4,kb4,ka7,kb7,kcat7,VA = can[:13]

    ka5,kb5,kcat5 = can[9:12]
    ka6,kb6,kcat6 = can[0:3]

    #Initial conditions
    L,Lp,K,P,LK,A,LpA,LpAK,LpAP,LpAPLp,LpAKL,LpP = Y

    sigma = 0.001

    dL = (kb1*LK) - (ka1*L*K) + (kcat5*LpAPLp) + (kb6*LpAKL) - ((VA/(2*sigma))*ka6*LpAK*L) + (kcat7*LpP)
    dLp = (kcat1*LK) + (kb2*LpA) - (ka2*Lp*A) + (kb5*LpAPLp) - ((VA/(2*sigma))*ka5*Lp*LpAP) + (kcat6*LpAKL) - (ka7*Lp*P) + (kb7*LpP)
    dK = (kb1*LK) - (ka1*L*K) + (kcat1*LK) + (kb3*LpAK) - (ka3*LpA*K)
    dP = (kb4*LpAP) - (ka4*LpA*P) - (ka7*Lp*P) + (kb7*LpP) + (kcat7*LpP)
    dLK = (ka1*L*K) - (kb1*LK) - (kcat1*LK)
    dA = (kb2*LpA) - (ka2*Lp*A)
    dLpA = (ka2*Lp*A) - (kb2*LpA) + (kb3*LpAK) - (ka3*LpA*K) + (kb4*LpAP) - (ka4*LpA*P)
    dLpAK = (ka3*LpA*K) - (kb3*LpAK) + (kb6*LpAKL) - ((VA/(2*sigma))*ka6*LpAK*L) + (kcat6*LpAKL)
    dLpAP = (ka4*LpA*P) - (kb4*LpAP) + (kb5*LpAPLp) - ((VA/(2*sigma))*ka5*LpAP*Lp) + (kcat5*LpAPLp)
    dLpAPLp = ((VA/(2*sigma))*ka5*LpAP*Lp) - (kb5*LpAPLp) - (kcat5*LpAPLp)
    dLpAKL = ((VA/(2*sigma))*ka6*LpAK*L) - (kb6*LpAKL) - (kcat6*LpAKL)
    dLpP = (ka7*Lp*P) - (kb7*LpP) - (kcat7*LpP)

    #Y[0],Y[1],Y[2],Y[3],Y[4],Y[5],Y[6],Y[7],Y[8],Y[9],Y[10],Y[11] = dL,dLp,dK,dP,dLK,dA,dLpA,dLpAK,dLpAP,dLpAPLp,dLpAKL,dLpP

    return([dL, dLp, dK, dP, dLK, dA, dLpA, dLpAK, dLpAP, dLpAPLp, dLpAKL, dLpP])	
    #return Y

In [10]:
t = np.linspace(0,100,num=10000)

candidate = List([0.5399439956871153,
 32.49731125365913,
 459.0713154241908,
 0.6703697205723796,
 98.55386776137301,
 66.94761309866233,
 187.88476784879947,
 0.732829082837601,
 0.5932803922080772,
 0.8833353263653272,
 79.8349667706061,
 98.51067194455857,
 1.21704450980531,
 8.973816043747753,
 0.6170991018130821,
 0.7934153177539267,
 2.6856696379362606])

initials = [candidate[13],0,candidate[14],
            candidate[15],0,candidate[16],0,0,
            0,0,0,0]

%timeit sol = odeint(oscillatorModelOde,initials,t,args=(candidate,))



111 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
sol

array([[8.97381604e+00, 0.00000000e+00, 6.17099102e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.94574768e+00, 2.19188299e-02, 6.11119205e-01, ...,
        3.52711780e-09, 1.07010152e-05, 4.14390236e-05],
       [8.91804870e+00, 4.90428423e-02, 6.11008743e-01, ...,
        1.04543559e-07, 8.35393493e-05, 1.35459839e-04],
       ...,
       [4.31660430e+00, 4.28310057e+00, 5.61027271e-01, ...,
        9.36161520e-02, 3.96539406e-02, 1.42345984e-02],
       [4.22896774e+00, 4.36805431e+00, 5.60767229e-01, ...,
        9.40326127e-02, 3.96781666e-02, 1.45090904e-02],
       [4.14225461e+00, 4.45215566e+00, 5.60539695e-01, ...,
        9.44506727e-02, 3.96712207e-02, 1.47803955e-02]])

In [9]:
from scipy.special import gamma, airy
y1_0 = 1.0 / 3**(2.0/3.0) / gamma(2.0/3.0)
y0_0 = -1.0 / 3**(1.0/3.0) / gamma(1.0/3.0)
y0 = [y0_0, y1_0]
t = np.arange(0, 4.0, 0.01)


In [13]:
from numba import jit
@jit
def RHS(y, t):
    y[0],y[1] = t*y[1],y[0]
    return y

%timeit odeint(RHS, y0, t)

101 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [21]:
initials = tf.constant([candidate[13],0.0,candidate[14],candidate[15],0.0,candidate[16],0.0,0.0,0.0,0.0,0.0,0.0])
candidate = tf.constant(candidate)


In [32]:
t_begin = tf.constant(0.0)
t_end = tf.constant(100.0)

In [33]:
sol = tfp.math.ode.BDF().solve(oscillatorModelOde, t_begin, initials, solution_times=tfp.math.ode.ChosenBySolver(t_end),constants={'can':candidate})